### image size : 224 x 224 (통일)
### teacher model : resnet50
### learning rate = 0.005
### 일단은 learning rate만 바꿔보면서 해보기

In [8]:
!pip install vit-pytorch

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import CIFAR100
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet50
from vit_pytorch.distill import DistillableViT, DistillWrapper

In [10]:
# 전처리를 위한 변환 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조정
    transforms.ToTensor(),  # 이미지를 Tensor로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 이미지 정규화
])

In [11]:
# CIFAR-100 데이터셋 불러오기
train_dataset = CIFAR100(root='./data', train=True, transform=transform, download=True)
test_dataset = CIFAR100(root='./data', train=False, transform=transform, download=True)

# 데이터 로더 설정
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [12]:
# 모델 생성
teacher = resnet50(pretrained=True)



    
student = DistillableViT(
    image_size=256,
    patch_size=32,
    num_classes=1000,
    dim=1024,
    depth=6,
    heads=8,
    mlp_dim=2048,
    dropout=0.1,
    emb_dropout=0.1
)

class DistillWrapper(nn.Module):
    def __init__(self, student_model, teacher_model, temperature, alpha, hard):
        super(DistillWrapper, self).__init__()
        self.student_model = student_model
        self.teacher_model = teacher_model
        self.temperature = temperature
        self.alpha = alpha
        self.hard = hard

    def forward(self, x):
        with torch.no_grad():
            teacher_outputs = self.teacher_model(x)
        student_outputs = self.student_model(x)
        return teacher_outputs, student_outputs

distiller = DistillWrapper(
    student_model=student,
    teacher_model=teacher,
    temperature=3,
    alpha=0.5,
    hard=False
)


In [13]:
# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
distiller = distiller.to(device)

# 손실 함수와 최적화 알고리즘 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(distiller.parameters(), lr=0.005)

In [14]:
def train(model, dataloader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        teacher_outputs, student_outputs = model(images)  # 수정된 부분
        _, predicted = torch.max(student_outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        loss = criterion(student_outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    accuracy = 100 * correct / total
    return running_loss, accuracy

def evaluate(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            teacher_outputs, student_outputs = model(images)  # 수정된 부분
            _, predicted = torch.max(student_outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            loss = criterion(student_outputs, labels)
            running_loss += loss.item()

    accuracy = 100 * correct / total
    return running_loss, accuracy


# 모델 학습 및 평가
num_epochs = 10

for epoch in range(num_epochs):
    print("Training step:", epoch+1)
    train_loss, train_accuracy = train(distiller, train_loader, criterion, optimizer)
    print("Evaluating step:", epoch+1)
    val_loss, val_accuracy = evaluate(distiller, test_loader, criterion)

    print("Epoch: {}/{}".format(epoch+1, num_epochs))
    print("Train Loss: {:.4f} | Train Accuracy: {:.2f}%".format(train_loss, train_accuracy))
    print("Val Loss: {:.4f} | Val Accuracy: {:.2f}%".format(val_loss, val_accuracy))
    print()


Training step: 1
Evaluating step: 1
Epoch: 1/10
Train Loss: 7189.5888 | Train Accuracy: 2.33%
Val Loss: 1375.1506 | Val Accuracy: 3.46%

Training step: 2
Evaluating step: 2
Epoch: 2/10
Train Loss: 6948.9828 | Train Accuracy: 2.56%
Val Loss: 1380.5375 | Val Accuracy: 2.87%

Training step: 3


KeyboardInterrupt: 